In [58]:
import pandas as pd

# Leitura da Base IPDM (Índice Paulistas de Desenvolvimento Municipal)
ipdm = pd.read_csv("arq_ipdm_painel_v2024.csv",
                   sep=';',
                   encoding='ISO-8859-1')

# Leitura da Base de códigos dos Municípios, fornecido pela Fundação SEADE
codigos = pd.read_csv(r"codigos_municipios_regioes.csv", sep=';',
                      encoding="ISO-8859-1")
codigos.drop(columns=['cod_ra', 'cod_ibge6', 'pop2020', 'porte_pop2020', 'cod_drs', 'drs'],
             axis=1, inplace=True)
codigos = codigos.iloc[1:, :]

In [71]:
merge_df = pd.DataFrame()

teste = ipdm[(ipdm['Ano'] == 2014) & (ipdm['Municipio'] == 'Adamantina')]
teste['Valor'] = teste['Valor'].str.replace(',', '.', regex=False).astype(float, errors='ignore')

for tipo in teste['Tipo'].unique():
    if tipo == 'IPDM':
        df_pivot = teste[teste['Tipo'] == tipo].pivot_table(
            
            index = ['cod_ibge', 'Municipio', 'Ano'],
            columns=['Indicador5', 'Indicador4', 'Indicador3', 'Indicador2', 'Indicador1'],
            values='Valor',
            aggfunc='first'
            
        ).reset_index()
        
        if tipo == 'IPDM':
            df_pivot.columns = ['cod_ibge', 'Municipio', 'Ano',
                                'IPDM']
            
        elif tipo == 'Escolaridade':
            df_pivot.columns = ['cod_ibge', 'Municipio', 'Ano',
                                'Taxa de atendimento escolar na faixa etária de 0 a 3 anos',
                                'Proporção média de alunos do 5º ano com proficiência em Língua Portuguesa e Matemática',
                                'Proporção média de alunos do 9º ano com proficiência em Língua Portuguesa e Matemática',
                                'Taxas de distorção idade-série no Ensino Médio',
                                'Indicador Escolaridade']
            
        elif tipo == 'Riqueza':
            df_pivot.columns = ['cod_ibge', 'Municipio', 'Ano', 
                        'Consumo anual de energia elétrica residencial', 
                        'Consumo anual de energia elétrica comercial', 
                        'Rendimento do trabalho formal', 
                        'Produto Interno Bruto per capita', 
                        'Indicador Riqueza']

        elif tipo == 'Longevidade':
            df_pivot.columns = ['cod_ibge', 'Municipio', 'Ano',
                                'Taxas de mortalidade infantil',
                                'Taxas de mortalidade perinatal',
                                'Taxas de mortalidade 15 a 39 anos',
                                'Taxas de mortalidade 60 a 69 anos',
                                'Indicador Longevidade']
            
        if merge_df.empty:
            merge_df = df_pivot

        else:
            
            merge_df = merge_df.merge(df_pivot, how='inner', on=['cod_ibge', 'Municipio', 'Ano'])
            

merge_df = merge_df.drop_duplicates()

C:\Users\muril\AppData\Local\Temp\ipykernel_21488\1758025552.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teste['Valor'] = teste['Valor'].str.replace(',', '.', regex=False).astype(float, errors='ignore')


In [72]:
merge_df

,cod_ibge,Municipio,Ano,IPDM
0,3500105,Adamantina,2014,0.548


In [ ]:
dict_bases_anos = {}
def processar_base_por_ano(ano, municipio, df):

    df_ano = df.copy()
    df_ano = df_ano[df_ano['Ano'] == ano]
    

    df_ano['Valor'] = df_ano['Valor'].str.replace(',', '.', regex=False).astype(float, errors='ignore')
    for tipo in df_ano['Tipo'].unique():
        df_tipo = df_ano[df_ano['Tipo'] == tipo]
        
        df_pivot = df_tipo.pivot_table(
            
            index = ['cod_ibge', 'Municipio', 'Ano'],
            columns=['Indicador5', 'Indicador4', 'Indicador3', 'Indicador2', 'Indicador1'],
            values='Valor',
            aggfunc='first'
        )

In [8]:
amostra = pd.read_csv("amostra.csv")
amostra

,Unnamed: 0,cod_ibge,Municipio,Valor,Ano,Tipo,Indicador1,Indicador2,Indicador3,Indicador4,Indicador5
0,3225,3500105,Adamantina,"2,42000319030148",2014,Riqueza,Consumo anual de energia elétrica residencial ...,,,,
1,6450,3500105,Adamantina,"8,97541371158392",2014,Riqueza,,"Consumo anual de energia elétrica comercial, s...",,,
2,9675,3500105,Adamantina,"1375,20557875883",2014,Riqueza,,,Rendimento do trabalho formal mais benefícios ...,,
3,12900,3500105,Adamantina,"53854,1008777062",2014,Riqueza,,,,Produto Interno Bruto per capita (R$ de 2022),
4,41925,3500105,Adamantina,",365",2014,Riqueza,,,,,Indicador Riqueza


In [9]:
teste_df = amostra.copy()
teste_df['Valor'] = teste_df['Valor'].str.replace(',', '.', regex=False).astype(float, errors='ignore')

df_pivot = teste_df.pivot_table(
    index=['cod_ibge', 'Municipio', 'Ano'],
    columns=['Indicador5', 'Indicador4', 'Indicador3', 'Indicador2', 'Indicador1'],
    values='Valor',
    aggfunc='first'
    
).reset_index()

df_pivot.columns = ['cod_ibge', 'Municipio', 'Ano', 
                    'Consumo anual de energia elétrica residencial', 
                    'Consumo anual de energia elétrica comercial', 
                    'Rendimento do trabalho formal', 
                    'Produto Interno Bruto per capita', 
                    'Indicador Riqueza']


In [10]:
df_pivot

,cod_ibge,Municipio,Ano,Consumo anual de energia elétrica residencial,Consumo anual de energia elétrica comercial,Rendimento do trabalho formal,Produto Interno Bruto per capita,Indicador Riqueza
0,3500105,Adamantina,2014,2.420003,8.975414,1375.205579,53854.100878,0.365
